In [1]:
from sklearn.model_selection import train_test_split
import os
import torch
import sys
from pathlib import Path

In [2]:
sys.path.append(os.path.abspath("..")) 

In [3]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
if not (project_root / 'src').exists():
    print(f"⚠️ Ostrzeżenie: Nie znaleziono katalogu 'src' w {project_root}")
    
print(f"Dodano do ścieżki: {project_root}")

Dodano do ścieżki: /Users/michaliwaniuk/fromscratch-transformer-classifier


In [4]:
import pandas as pd
from src.textclf_transformer import * 

In [5]:
from datasets import load_dataset, concatenate_datasets

ds = load_dataset("imdb")

merged = concatenate_datasets([ds["test"], ds["train"]])
df = pd.DataFrame(merged)

In [7]:
df.columns = ['review', 'sentiment']

In [9]:
df["review"] = df["review"].str.replace(r"<br\s*/?>", "", regex=True)

In [17]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    stratify=df["sentiment"],
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,          # połowa z 10% = 5%
    random_state=42,
    stratify=temp_df["sentiment"],
)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 40000 | Val: 5000 | Test: 5000


In [18]:
tok = WordPieceTokenizerWrapper()
path = Path("../../src/textclf_transformer/tokenizer/BERT_original")
tok.load(tokenizer_dir=path)

In [19]:
tokens_train = tok.encode_pandas(train_df, 'review',
                                  max_length=512, 
                                  label_col='sentiment'
                                 )
tokens_val = tok.encode_pandas(val_df, 'review',
                                max_length=512,
                                label_col='sentiment')
tokens_test = tok.encode_pandas(test_df, 'review',
                                 max_length=512,
                                 label_col='sentiment')

In [20]:
print(tok.tokenizer.decode(tokens_train[0][0]))
print(tokens_train[0][0])  # label

[CLS] i'd love to give kolchak a higher rating but the show quickly went from scary / suspenseful to silly. abc's fault. they moved the show to friday nights at 8 : 00 p. m., then known as the " family hour ". never should have been on fridays in the first place. i was a sophomore in high school and loved the early episodes! it was first up against police woman on nbc. abc had huge problems with friday nights. bad season for them overall until barney miller, baretta, and swat debuted in january of'75. kolchak should have been a hit. darren mcgavin begged to get out of his contract to end the show. too bad the writing wasn't up to richard matheson's in the original tv movies. still, mcgavin made kolchak his own, as actors can do. jackie gleason as ralph kramden and caroll o'connor as archie bunker come to mind. that ins set with the manual typewriters and clacking teletypes seems quaint and ancient today, yet that was part of the appeal. they were very lucky to have simon oakland repris

In [21]:
save_dir = Path("../../data/tokenized")
save_dir.mkdir(parents=True, exist_ok=True)

torch.save(tokens_train, save_dir / "imdb_train.pt")
torch.save(tokens_val, save_dir / "imdb_val.pt")
torch.save(tokens_test, save_dir / "imdb_test.pt")